# Workshop DL01 - Intro to Deep Learning

### Preamble

#### Installing TensorFlow
1. Open the Anaconda3 Terminal
2. Execute `pip install --upgrade tensorflow`

#### Getting the MNIST Data
1. Go to https://www.kaggle.com/c/digit-recognizer/data
2. Download "train.csv" and "test.csv", put these into the same folder as this notebook

# Deep Neural Network Image Classifier

In [113]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [114]:
# read-in raw data
data = pd.read_csv("datasets/mnist_train.csv")

# mask for splitting data into train and test sets
mask = np.random.rand(len(data)) < 0.90

# training data, features (X) and labels (y)
train_X = data[mask].drop("label", axis=1)
train_y = data.label[mask]

# test data, features (X) and labels (y)
test_X = data[~mask].drop("label", axis=1)
test_y = data.label[~mask]

In [115]:
# helper functions for defining weights/biases in tensorflow
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1), shape)

In [116]:
# (1) define network architecture
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="features")
    y = tf.placeholder(tf.int64, [None], name="labels")

with tf.name_scope("fc-layer-1"):
    w1 = weight_variable([784, 500])
    b1 = bias_variable([500])
    fc1 = tf.nn.relu(tf.matmul(x, w1) + b1)

with tf.name_scope("fc-layer-2"):
    w2 = weight_variable([500, 500])
    b2 = bias_variable([500])
    fc2 = tf.nn.relu(tf.matmul(fc1, w2) + b2)
    
with tf.name_scope("output"):
    w3 = weight_variable([500, 10])
    b3 = bias_variable([10])
    y_ = tf.matmul(fc2, w3) + b3

In [117]:
# (2) define loss function
with tf.name_scope("loss"):
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(
        labels=y, logits=y_
    )

In [118]:
# (3) define optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(
        cross_entropy
    )

In [119]:
# aside: evaluation operations
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(y_, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
# (4) train network
with tf.Session() as sess:
    
    # initialize all our operations
    sess.run(tf.global_variables_initializer())
    
    # train the network
    for epoch in range(20):
        
        # Note: usually you have to chunk through the data in another
        #       for loop; the data here is small though, so you might not
        #       have to.
        _, epoch_loss = sess.run([optimizer, cross_entropy],
                feed_dict={x: train_X, y: train_y})
        
        print("Epoch " + str(epoch) + " completed, loss = "
              + str(epoch_loss))
        
    # evaluate on the test data
    print("==================================================")
    score = sess.run([accuracy],
                feed_dict={x: test_X, y: test_y})
    print("Test accuracy = " + str(score))

Epoch 0 completed, loss = 569.14136
Epoch 1 completed, loss = 2968.1665
